
## 📌 Pembagian Tugas (3 Anggota)
Setiap anggota berkontribusi dalam tahap **Modelling dan Evaluation** dengan skenario berbeda.

### Model Supervised Learning:
- **Anggota 1**: Split Dataset 70:30
- **Anggota 2**: Split Dataset 80:20
- **Anggota 3**: Split Dataset 65:35

### Model Unsupervised Learning:
- **Anggota 1**: 3 Cluster
- **Anggota 2**: 4 Cluster
- **Anggota 3**: 5 Cluster


#import


In [4]:
pip install optuna darts


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
from flask import Flask, jsonify, request
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler
import torch
from flask import Flask, jsonify, request
import optuna
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from darts.metrics import mae, rmse

#MODELING

do not run this


In [6]:
import pandas as pd

# Load new dataset
df = pd.read_csv("dataset_konsumsi.csv")
print(df.head())

   periode_data  tahun          wilayah        sektor  total_konsumsi
0          2022   2022    Jakarta Pusat  Rumah Tangga          140.36
1          2022   2022    Jakarta Utara  Rumah Tangga          233.10
2          2022   2022    Jakarta Barat  Rumah Tangga          318.28
3          2022   2022  Jakarta Selatan  Rumah Tangga          291.72
4          2022   2022    Jakarta Timur  Rumah Tangga          400.80


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   periode_data    6 non-null      int64  
 1   tahun           6 non-null      int64  
 2   wilayah         6 non-null      object 
 3   sektor          6 non-null      object 
 4   total_konsumsi  6 non-null      float64
dtypes: float64(1), int64(2), object(2)
memory usage: 372.0+ bytes


In [10]:
total_columns = [col for col in df.columns if col.endswith('- total')]

In [13]:
# Filter the dataset to include only the year and '-total' columns
filtered_data = df[['tahun'] + total_columns]


In [12]:
total_columns

[]

In [15]:
melted_data = pd.melt(
    filtered_data,
    id_vars=["tahun"],
    var_name="Fuel_Type",
    value_name="Consumption"
)

# Convert 'tahun' to datetime (kalau perlu)
melted_data['tahun'] = pd.to_datetime(melted_data['tahun'], format='%Y')


# Display the melted data
melted_data.head()

,tahun,Fuel_Type,Consumption


In [16]:
melted_data['Consumption'] = melted_data['Consumption'].replace({',': ''}, regex=True)
melted_data['Consumption'] = pd.to_numeric(melted_data['Consumption'], errors='coerce')

In [17]:
melted_data

,tahun,Fuel_Type,Consumption


In [29]:
# Cek data asli sebelum melt
df = pd.read_csv("dataset_konsumsi.csv")
print(df.head())         # pastikan ini tidak kosong
print(df.columns)        # cek nama kolom

# Setelah melt
melted_data = df.melt(...)  # isi sesuai kode kamu
print(melted_data.head())   # pastikan hasil melt masuk akal


   periode_data  tahun          wilayah        sektor  total_konsumsi
0          2022   2022    Jakarta Pusat  Rumah Tangga          140.36
1          2022   2022    Jakarta Utara  Rumah Tangga          233.10
2          2022   2022    Jakarta Barat  Rumah Tangga          318.28
3          2022   2022  Jakarta Selatan  Rumah Tangga          291.72
4          2022   2022    Jakarta Timur  Rumah Tangga          400.80
Index(['periode_data', 'tahun', 'wilayah', 'sektor', 'total_konsumsi'], dtype='object')


KeyError: 'The following id_vars or value_vars are not present in the DataFrame: [Ellipsis]'

In [25]:
print(series)
print(series.pd_dataframe().head())  # Lihat isi aslinya


<TimeSeries (DataArray) (tahun: 0, component: 1, sample: 1)> Size: 0B
array([], shape=(0, 1, 1), dtype=float64)
Coordinates:
  * tahun      (tahun) datetime64[ns] 0B 
  * component  (component) <U11 44B 'Consumption'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None


AttributeError: 'TimeSeries' object has no attribute 'pd_dataframe'

In [28]:
print(melted_data.info())
print(melted_data.head())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 0 entries
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Fuel_Type    0 non-null      object 
 1   Consumption  0 non-null      float64
 2   Index        0 non-null      int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 0.0+ bytes
None
Empty DataFrame
Columns: [Fuel_Type, Consumption, Index]
Index: []


In [27]:
melted_data.reset_index(inplace=True)

# Cek dan siapkan kolom waktu dan nilai
melted_data['tahun'] = pd.to_datetime(melted_data['tahun'], format='%Y')
melted_data = melted_data.sort_values('tahun')
melted_data.set_index('tahun', inplace=True)

print(melted_data.head())  # pastikan tidak kosong

# Ubah ke TimeSeries Darts
series = TimeSeries.from_dataframe(melted_data, value_cols="Consumption", freq="Y")
print(series)  # harus ada data

# Lanjut normalisasi
scaler = Scaler()
series_scaled = scaler.fit_transform(series)

# Step 5: Train/Test split (using last 42 values for validation)
train, val = series_scaled[:-1], series_scaled[-1:]

# Objective function for Optuna
def objective(trial):
    # Hyperparameter tuning
    input_chunk_length = trial.suggest_int('input_chunk_length', 24, 120)
    output_chunk_length = trial.suggest_int('output_chunk_length', 6, 24)
    n_epochs = trial.suggest_int('n_epochs', 100, 500)
    batch_size = trial.suggest_int('batch_size', 16, 64)

    # Step 6: N-BEATS model with parameters from Optuna
    model = NBEATSModel(
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        n_epochs=n_epochs,
        random_state=42,
        batch_size=batch_size
    )

    # Step 7: Fit the model
    model.fit(train, verbose=False)

    # Step 8: Evaluate the model using validation data
    forecast = model.predict(n=len(val))
    # Calculate the Mean Absolute Percentage Error (MAPE) as a performance metric
    mape_value = np.mean(np.abs((forecast.values() - val.values()) / val.values())) * 100
    return mape_value

# Step 9: Set up Optuna study and optimize
study = optuna.create_study(direction='minimize')  # We want to minimize the MAPE
study.optimize(objective, n_trials=50)  # Number of trials (you can increase this for more thorough search)

# Step 10: Display the best parameters found by Optuna
print("Best trial:")
best_trial = study.best_trial
print(f"  Value (MAPE): {best_trial.value}")
print(f"  Params: {best_trial.params}")

# Step 11: Train the model with the best parameters
best_params = best_trial.params
best_model = NBEATSModel(
    input_chunk_length=best_params['input_chunk_length'],
    output_chunk_length=best_params['output_chunk_length'],
    n_epochs=best_params['n_epochs'],
    batch_size=best_params['batch_size'],
    random_state=42
)

best_model.fit(train, verbose=True)

# Step 12: Forecasting using the best model
forecast = best_model.predict(n=7)

# Step 13: Plotting the results
series_scaled.plot(label="Actual")
forecast.plot(label="Forecast")
plt.xlabel("Index")
plt.ylabel("Consumption")
plt.title("Fuel Consumption Forecast using Optimized N-BEATS")
plt.legend()
plt.show()


Empty DataFrame
Columns: [Fuel_Type, Consumption, Index]
Index: []
<TimeSeries (DataArray) (tahun: 0, component: 1, sample: 1)> Size: 0B
array([], shape=(0, 1, 1), dtype=float64)
Coordinates:
  * tahun      (tahun) datetime64[ns] 0B 
  * component  (component) <U11 44B 'Consumption'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None


c:\Users\TESTYA BIHAQQIS\AppData\Local\Programs\Python\Python311\Lib\site-packages\darts\timeseries.py:5248: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  resampled_time_index = resampled_time_index.asfreq(freq)


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
param = {'input_chunk_length': 115, 'output_chunk_length': 13, 'n_epochs': 115, 'batch_size': 30}
best_model = NBEATSModel(
    **param
)

best_model.fit(train, verbose=True)

# Step 12: Forecasting using the best model
forecast = best_model.predict(n=42)
# Step 13: Plotting the results
series_scaled.plot(label="Actual")
forecast.plot(label="Forecast")
plt.xlabel("Index")
plt.ylabel("Consumption")
plt.title("Fuel Consumption Forecast using Optimized N-BEATS")
plt.legend()
plt.show()

print(f"MAE: {mae(val, forecast)}")
print(f"RMSE: {rmse(val, forecast)}")


NameError: name 'train' is not defined

In [ ]:
best_model.save("nbeats_model.pth")

In [ ]:
# Step 1: Prepare Data (use index as time)
melted_data['Index'] = melted_data.index  # Adding index as a column for TimeSeries

# Step 2: Convert to TimeSeries using the index as time
series = TimeSeries.from_dataframe(melted_data, time_col="Index", value_cols="Consumption")
# Step 4: Normalization
scaler = Scaler()
series_scaled = scaler.fit_transform(series)

NameError: name 'melted_data' is not defined

#API IS FROM HERE

In [ ]:
app = Flask(__name__)

# Load the trained model
def load_model():
    best_model = NBEATSModel.load("nbeats_model.pth")
    return best_model

# Prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the data from the request
        data = request.get_json()

        # Extract necessary input from the request
        input_data = np.array(data["input"]).reshape(-1, 1)  # Input data
        n_predictions = data.get("n", 7)
        input_series = TimeSeries.from_values(input_data)
        scaler = Scaler()
        input_series_scaled = scaler.fit_transform(input_series)

        # Get the model and make the prediction
        model = load_model()
        forecast = model.predict(n=n_predictions)  # Predict `n_predictions` future values

        # Inverse transform the forecast
        forecast_values = scaler.inverse_transform(forecast.values())

        # Return the predicted values as a JSON response
        return jsonify({'prediction': forecast_values.tolist()})

    except Exception as e:
        return jsonify({'error': str(e)})

# Run the Flask application
if __name__ == '__main__':
    # Print the URL of the API after the app starts
    print("Starting the Flask API...")
    print(f"API is running at: http://127.0.0.1:5000/predict")
    app.run(debug=True)

c:\Users\TESTYA BIHAQQIS\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


API is running at: http://127.0.0.1:5000/predict
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\TESTYA BIHAQQIS\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 🧩 Supervised Learning - Anggota 1 (Split 70:30)

In [ ]:
ratio = 0.7
train_size = int(ratio * len(series_scaled))
train, val = series_scaled[:train_size], series_scaled[train_size:]

model = NBEATSModel(input_chunk_length=24, output_chunk_length=len(val))
model.fit(train, verbose=False)
forecast = model.predict(len(val))

print("Anggota 1 - Split 70:30")
print(f"MAPE : {mape(val, forecast):.2f}")
print(f"MAE  : {mae(val, forecast):.2f}")
print(f"RMSE : {rmse(val, forecast):.2f}")

val.plot(label="actual")
forecast.plot(label="forecast")
plt.title("Forecast - Anggota 1 (70:30)")
plt.legend()
plt.show()

### 🧩 Supervised Learning - Anggota 2 (Split 80:20)

In [ ]:
ratio = 0.8
train_size = int(ratio * len(series_scaled))
train, val = series_scaled[:train_size], series_scaled[train_size:]

model = NBEATSModel(input_chunk_length=24, output_chunk_length=len(val))
model.fit(train, verbose=False)
forecast = model.predict(len(val))

print("Anggota 2 - Split 80:20")
print(f"MAPE : {mape(val, forecast):.2f}")
print(f"MAE  : {mae(val, forecast):.2f}")
print(f"RMSE : {rmse(val, forecast):.2f}")

val.plot(label="actual")
forecast.plot(label="forecast")
plt.title("Forecast - Anggota 2 (80:20)")
plt.legend()
plt.show()

### 🧩 Supervised Learning - Anggota 3 (Split 65:35)

In [ ]:
ratio = 0.65
train_size = int(ratio * len(series_scaled))
train, val = series_scaled[:train_size], series_scaled[train_size:]

model = NBEATSModel(input_chunk_length=24, output_chunk_length=len(val))
model.fit(train, verbose=False)
forecast = model.predict(len(val))

print("Anggota 3 - Split 65:35")
print(f"MAPE : {mape(val, forecast):.2f}")
print(f"MAE  : {mae(val, forecast):.2f}")
print(f"RMSE : {rmse(val, forecast):.2f}")

val.plot(label="actual")
forecast.plot(label="forecast")
plt.title("Forecast - Anggota 3 (65:35)")
plt.legend()
plt.show()

### 🧩 Unsupervised Learning - Anggota 1 (3 Cluster)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster_3'] = kmeans.fit_predict(features)

sns.scatterplot(x='hour', y='value', hue='cluster_3', data=df, palette='Set1')
plt.title("Clustering - Anggota 1 (3 Cluster)")
plt.legend(title='Cluster')
plt.show()

### 🧩 Unsupervised Learning - Anggota 2 (4 Cluster)

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)
df['cluster_4'] = kmeans.fit_predict(features)

sns.scatterplot(x='hour', y='value', hue='cluster_4', data=df, palette='Set2')
plt.title("Clustering - Anggota 2 (4 Cluster)")
plt.legend(title='Cluster')
plt.show()

### 🧩 Unsupervised Learning - Anggota 3 (5 Cluster)

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster_5'] = kmeans.fit_predict(features)

sns.scatterplot(x='hour', y='value', hue='cluster_5', data=df, palette='Set3')
plt.title("Clustering - Anggota 3 (5 Cluster)")
plt.legend(title='Cluster')
plt.show()